In [1]:
import pymysql
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import sklearn
import pickle
import warnings

warnings.filterwarnings("ignore")

# === 1. MySQL Connection Setup ===
# IMPORTANT: Update the 'password' and 'host' if they are different for your setup.
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='varun@07',
    autocommit=True
)

cursor = connection.cursor()

# === 2. Create "TaxiData" Database ===
# A new database is created specifically for this data.
cursor.execute("CREATE DATABASE IF NOT EXISTS TaxiData")
cursor.execute("USE TaxiData")

# === 3. Load the CSV File ===
# The pandas library is used to read the taxi_trip_pricing.csv file into a DataFrame.
try:
    df_taxi = pd.read_csv('taxi_trip_pricing.csv')
    print("✅ CSV file loaded successfully into a pandas DataFrame!")
except FileNotFoundError:
    print("Error: The file 'taxi_trip_pricing.csv' was not found.")
    exit()

# === 4. Handle NaN values ===
# Replace NaN values with None to prevent "nan can not be used with MySQL" errors.
# We iterate through columns and replace NaN values based on the column's dtype.
for col in df_taxi.columns:
    if df_taxi[col].dtype == np.float64 or df_taxi[col].dtype == np.int64:
        df_taxi[col] = df_taxi[col].replace({np.nan: None})
    else:
        df_taxi[col] = df_taxi[col].replace({np.nan: ''})
print("✅ NaN values handled and replaced.")

# === 5. Create Table Automatically ===
# This function automatically generates a CREATE TABLE SQL statement
# based on the DataFrame's columns and data types.
def create_table_from_df(df, table_name):
    # Prepare a list of column names and their inferred SQL data types.
    cols = []
    for col in df.columns:
        # Replace spaces or invalid characters in column names with underscores.
        sql_col_name = f"`{col.replace(' ', '_').replace('.', '_').replace('(', '_').replace(')', '_')}`"
        
        dtype = df[col].dtype
        if 'int' in str(dtype) or 'float' in str(dtype):
            sql_type = 'FLOAT'
        else:
            # All other types are treated as VARCHAR. A size of 255 is a safe default.
            sql_type = 'VARCHAR(255)'
        cols.append(f"{sql_col_name} {sql_type}")
    
    columns_sql = ", ".join(cols)
    create_sql = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns_sql})"
    cursor.execute(create_sql)
    print(f"✅ Table '{table_name}' created successfully!")

# Create the table for the taxi data
create_table_from_df(df_taxi, 'taxi_trips')

# === 6. Insert Data into Tables (using executemany for speed) ===
# This function handles the insertion of data from the DataFrame into the SQL table.
def insert_data(df, table_name):
    # The NaN values have already been handled in a previous step.
    
    # Generate the column list for the INSERT statement.
    cols = ",".join([f"`{col.replace(' ', '_').replace('.', '_').replace('(', '_').replace(')', '_')}`" for col in df.columns])
    
    # Create placeholders for each column to prevent SQL injection.
    placeholders = ",".join(["%s"] * len(df.columns))
    
    insert_sql = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"
    
    # Convert the DataFrame to a list of tuples for batch insertion.
    data = [tuple(row) for row in df.itertuples(index=False, name=None)]
    
    # Execute the batch insert.
    cursor.executemany(insert_sql, data)
    print(f"✅ {len(data)} rows inserted into '{table_name}' successfully!")

# Insert the data into the 'taxi_trips' table
insert_data(df_taxi, 'taxi_trips')

# === 7. Machine Learning Workflow ===
print("\n--- Starting Machine Learning Workflow ---")

# Connect to the new database to load the data for the ML model
connection.close() # Close the previous connection
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='varun@07',
    database='taxidata'
)

# Load data directly from the newly created SQL table
df = pd.read_sql("SELECT * FROM taxi_trips", connection)
connection.close()

# Handle missing values for the ML model
# Numeric columns: fill with the mean
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
for col in numeric_cols:
    if df[col].isnull().any():
        df[col].fillna(df[col].mean(), inplace=True)
        
# Categorical columns: fill with a placeholder string
categorical_cols = df.select_dtypes(include='object').columns.tolist()
for col in categorical_cols:
    df[col].fillna('Unknown', inplace=True)

# Define X (features) and y (target variable)
X = df.drop('Trip_Price', axis=1)
y = df['Trip_Price']

# Identify categorical columns for one-hot encoding
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# === 8. Preprocessing and Modeling Pipeline ===
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Define Models to Compare
models = {
    "GradientBoosting": GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42),
    "LinearRegression": LinearRegression()
}

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === 9. Evaluate All Models ===
best_model_name = None
best_score = -np.inf
best_pipeline = None

for name, reg in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', reg)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False) # RMSE
    
    print(f"\n📊 {name} Results:")
    print(f"R² Score: {r2:.4f}")
    print(f"RMSE: {rmse:.2f}")
    
    if r2 > best_score:
        best_score = r2
        best_model_name = name
        best_pipeline = pipeline

# === 10. Save Best Model using Pickle ===
model_filename = "taxi_pricing_best_model.pkl"
with open(model_filename, "wb") as f:
    pickle.dump((best_pipeline, sklearn.__version__), f)

print(f"\n✅ Best Model: {best_model_name} (R² = {best_score:.4f}) saved successfully as {model_filename}")


✅ CSV file loaded successfully into a pandas DataFrame!
✅ NaN values handled and replaced.
✅ Table 'taxi_trips' created successfully!
✅ 1000 rows inserted into 'taxi_trips' successfully!

--- Starting Machine Learning Workflow ---

📊 GradientBoosting Results:
R² Score: 0.9645
RMSE: 5.86

📊 RandomForest Results:
R² Score: 0.9844
RMSE: 3.89

📊 LinearRegression Results:
R² Score: 0.8019
RMSE: 13.85

✅ Best Model: RandomForest (R² = 0.9844) saved successfully as taxi_pricing_best_model.pkl
